In [1]:
#basic imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score,classification_report

In [2]:
df = pd.read_csv('/kaggle/input/cleaned-2048-sample/Cleaned_2048_sample.csv')

In [3]:
device = torch.device("cuda")
torch.cuda.empty_cache()

In [4]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [5]:
df.dropna(inplace=True)

In [6]:
df['Move'] = df['Move'].astype('float16')
cols = list(df.columns)[:-1]
for col in cols:
    df[col] = df[col].astype('int16')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3062429 entries, 0 to 3062438
Data columns (total 17 columns):
 #   Column   Dtype  
---  ------   -----  
 0   tile_0   int16  
 1   tile_1   int16  
 2   tile_2   int16  
 3   tile_3   int16  
 4   tile_4   int16  
 5   tile_5   int16  
 6   tile_6   int16  
 7   tile_7   int16  
 8   tile_8   int16  
 9   tile_9   int16  
 10  tile_10  int16  
 11  tile_11  int16  
 12  tile_12  int16  
 13  tile_13  int16  
 14  tile_14  int16  
 15  tile_15  int16  
 16  Move     float16
dtypes: float16(1), int16(16)
memory usage: 122.7 MB


In [8]:
#board state
for i in range(len(list(df.iloc[1000]))-1):
    element = list(df.iloc[1000])[i]
    if element == 0.0:
        print("0.0",end=" ")
    else:
        print(2 ** element,end=" ")
    if (i+1) % 4==0:
        print("\n")

256.0 512.0 1024.0 2048.0 

128.0 4.0 8.0 4.0 

0.0 0.0 8.0 2.0 

0.0 0.0 2.0 4.0 



### 0=up, 1=right, 2=down, 3-left

In [9]:
df['Move'].value_counts()

Move
0.0    1599125
3.0     734685
1.0     728494
2.0        125
Name: count, dtype: int64

In [10]:
X = df.drop('Move',axis=1)
y = df['Move']

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y)

### Basic ML models

In [12]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [13]:
X_train = torch.Tensor(X_train.values).to(device)
X_test = torch.Tensor(X_test.values).to(device)
y_train = torch.LongTensor(y_train.values).to(device)

In [14]:
class Model(nn.Module):
    def __init__(self,in_features,h1,h2,h3,h4,h5,out_features):
        super(Model,self).__init__()
        self.fc1 = nn.Linear(in_features,h1)
        self.bn1 = nn.BatchNorm1d(h1)
        self.fc2 = nn.Linear(h1,h2)
        self.bn2 = nn.BatchNorm1d(h2)
        self.fc3 = nn.Linear(h2,h3)
        self.bn3 = nn.BatchNorm1d(h3)
        self.fc4 = nn.Linear(h3,h4)
        self.bn4 = nn.BatchNorm1d(h4)
        self.fc5 = nn.Linear(h4,h5)
        self.bn5 = nn.BatchNorm1d(h5)
        self.out = nn.Linear(h5,out_features)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = F.relu(self.fc2(x))
        x = self.bn2(x)
        x = F.relu(self.fc3(x))
        x = self.bn3(x)
        x = F.relu(self.fc4(x))
        x = self.bn4(x)
        x = F.relu(self.fc5(x))
        x = self.bn5(x)
        x = self.out(x)
        return x  

In [15]:
model = Model(in_features=16,h1=256,h2=256,h3=64,h4=32,h5=16,out_features=4).to(device)

In [16]:
model

Model(
  (fc1): Linear(in_features=16, out_features=256, bias=True)
  (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=256, out_features=64, bias=True)
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc4): Linear(in_features=64, out_features=32, bias=True)
  (bn4): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc5): Linear(in_features=32, out_features=16, bias=True)
  (bn5): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (out): Linear(in_features=16, out_features=4, bias=True)
)

In [17]:
num_epochs = 2500
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

In [18]:
for epoch in range(num_epochs):
        y_pred = model(X_train)  # Forward pass
        loss = criterion(y_pred,y_train)  # Compute loss

        # Print loss every 50 epochs
        if (epoch+1)%50==0:
            print(f"epoch: {epoch+1}  loss: {loss.item()}")

        optimizer.zero_grad()  # Zero out gradients
        loss.backward()  # Backward pass
        optimizer.step()  # Update parameters

epoch: 50  loss: 0.9131903648376465
epoch: 100  loss: 0.8510419130325317
epoch: 150  loss: 0.7938498258590698
epoch: 200  loss: 0.7543414235115051
epoch: 250  loss: 0.7307723164558411
epoch: 300  loss: 0.7039909362792969
epoch: 350  loss: 0.6899550557136536
epoch: 400  loss: 0.6698487997055054
epoch: 450  loss: 0.6635820269584656
epoch: 500  loss: 0.6489924788475037
epoch: 550  loss: 0.6393735408782959
epoch: 600  loss: 0.6445472836494446
epoch: 650  loss: 0.623632550239563
epoch: 700  loss: 0.6270760893821716
epoch: 750  loss: 0.6154249906539917
epoch: 800  loss: 0.6159639358520508
epoch: 850  loss: 0.6001317501068115
epoch: 900  loss: 0.6052342057228088
epoch: 950  loss: 0.5943282246589661
epoch: 1000  loss: 0.5932260751724243
epoch: 1050  loss: 0.5884150862693787
epoch: 1100  loss: 0.5869249105453491
epoch: 1150  loss: 0.5862035155296326
epoch: 1200  loss: 0.5892390012741089
epoch: 1250  loss: 0.5763721466064453
epoch: 1300  loss: 0.5795552134513855
epoch: 1350  loss: 0.582969486713

In [19]:
with torch.no_grad():
    y_eval = model.forward(X_test)
    loss_eval = criterion(y_eval,torch.LongTensor(y_test.values).to(device))

In [20]:
print(loss_eval)

tensor(0.6011, device='cuda:0')


In [21]:
y_eval = torch.argmax(y_eval,axis=1)

In [22]:
print(accuracy_score(y_eval.cpu().numpy(),y_test))

0.7398224678947973


In [23]:
# Save only the model's weights (state dictionary)
torch.save(model.state_dict(), "model_weights.pth")

# Move the weights file to the Kaggle working directory for download
import shutil
shutil.move("model_weights.pth", "/kaggle/working/model_weights.pth")

'/kaggle/working/model_weights.pth'